In [ ]:
# составим матрицу R пользователей (user-based)

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [2]:
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

In [3]:
user_data = pd.read_sql('SELECT * FROM public.user_data', con = engine)
posts = pd.read_sql('SELECT * FROM public.post_text_df', con = engine)

In [26]:
total_users_feed = pd.read_sql("""SELECT COUNT(DISTINCT user_id) FROM public.feed_data WHERE action = 'view'""", con = engine)

In [3]:
# берем для начала по ___ действий каждого пользователя
feed_data_bs_collab = pd.read_sql("""WITH user_actions AS (
                                        SELECT *,
                                            ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY timestamp DESC) as action_rank
                                        FROM public.feed_data
                                        WHERE action = 'view'
                                    )
                                    SELECT user_id, post_id, timestamp, target
                                    FROM user_actions
                                    WHERE action_rank <= 20
                                    -- Опционально: LIMIT для общего количества строк
                                    LIMIT 20000000""", con = engine)

In [4]:
#print(user_data.user_id.nunique())
feed_data_bs_collab[feed_data_bs_collab['user_id'] == 200]
print(feed_data_bs_collab.user_id.nunique())
print(feed_data_bs_collab.target.value_counts())
print(f'соотношение между 0 и 1 в target: {(2911731 - 352369) / 2911731}')

163205
target
0    2911731
1     352369
Name: count, dtype: int64
соотношение между 0 и 1 в target: 0.8789829829747322


In [54]:
feed_data_bs_collab

,user_id,post_id,timestamp,target
0,200,1773,2021-12-29 15:24:59,0
1,200,2213,2021-12-29 15:24:31,1
2,200,1122,2021-12-29 15:23:54,0
3,200,1362,2021-12-29 15:23:29,0
4,200,1541,2021-12-29 15:21:53,0
...,...,...,...,...
3264095,168552,1823,2021-12-23 10:29:21,0
3264096,168552,5258,2021-12-23 10:29:11,0
3264097,168552,7063,2021-12-23 10:28:48,0
3264098,168552,6793,2021-12-23 10:27:50,0


In [55]:
# Ячейка посвящена преобразованиям над текстом
import nltk
import re
from nltk.corpus import stopwords
nltk.download("stopwords") # поддерживает удаление стоп-слов
nltk.download('punkt') # делит текст на список предложений
nltk.download('wordnet')

# Лемматизация и стемминг
nltk.download('punkt_tab')

new_text = []
lemmatize = nltk.WordNetLemmatizer()
for i in posts.text:
    text = i.lower()
    text = re.sub("[^a-zA-Z]"," ", text)
    tokens = nltk.word_tokenize(text, language = "english")
    lemmatized_tokens = [lemmatize.lemmatize(word) for word in tokens]

    processed_text = ' '.join(lemmatized_tokens)
    new_text.append(processed_text)

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

#posts = posts.dropna()
tfidf = TfidfVectorizer(stop_words='english')
train_tfidf = tfidf.fit_transform(new_text)

tfidf_df = pd.DataFrame(
    train_tfidf.toarray(),
    columns = tfidf.get_feature_names_out()
)

# Не центруем - матрица разряжена!
# Выбираем подход -> TruncatedSVD
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components = 50)
SVD_dataset = svd.fit_transform(tfidf_df)

SVD_dataset = pd.DataFrame(SVD_dataset,
                           columns = [f'Компонента {i+1}' for i in range(SVD_dataset.shape[1])]
                          )

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vladimirkazakov/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vladimirkazakov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vladimirkazakov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/vladimirkazakov/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [56]:
SVD_dataset

,Компонента 1,Компонента 2,Компонента 3,Компонента 4,Компонента 5,Компонента 6,Компонента 7,Компонента 8,Компонента 9,Компонента 10,...,Компонента 41,Компонента 42,Компонента 43,Компонента 44,Компонента 45,Компонента 46,Компонента 47,Компонента 48,Компонента 49,Компонента 50
0,0.116060,-0.186410,-0.049454,-0.000523,-0.101014,-0.133805,-0.202203,-0.065532,-0.074325,-0.106313,...,0.002179,0.010628,0.042705,-0.004414,0.056644,-0.017693,0.029219,0.022165,-0.006796,0.063412
1,0.144863,-0.213865,-0.057816,-0.049949,0.040699,-0.041332,-0.027860,0.008698,-0.021705,0.015439,...,-0.023110,0.012462,-0.065136,-0.045523,-0.016480,0.035292,0.042335,-0.056266,0.060261,0.010389
2,0.124525,-0.147405,-0.033435,0.010892,-0.114615,-0.132843,-0.157761,-0.064839,-0.007534,-0.029431,...,-0.020743,-0.013039,-0.074508,0.012095,0.023199,-0.001687,-0.046311,-0.054737,0.038111,0.010659
3,0.099042,-0.150082,-0.023254,-0.003143,-0.075856,-0.127150,-0.110323,-0.029163,0.039768,0.046185,...,0.029200,-0.032290,0.013453,-0.015176,0.033650,-0.017317,0.012739,0.004808,0.045596,0.054767
4,0.064908,-0.077954,-0.022647,0.000434,-0.032228,-0.038103,-0.007236,0.005751,0.010157,0.035557,...,-0.000041,0.038725,-0.003999,-0.024108,0.021070,-0.007482,0.012024,-0.002668,-0.006443,0.000474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,0.254660,0.124200,0.017228,-0.143201,-0.033977,0.015009,0.008859,-0.018523,0.056992,-0.044557,...,0.024850,0.032919,0.004154,-0.012431,-0.000925,-0.066557,-0.037416,0.030056,0.036888,0.023625
7019,0.209761,0.108771,0.014662,-0.092210,-0.014198,-0.046055,-0.001134,0.022627,-0.030387,0.000598,...,-0.003589,-0.003201,-0.010161,0.020312,0.002428,-0.012027,-0.010665,-0.006095,-0.021190,-0.014359
7020,0.138492,0.056343,0.003408,0.115938,0.076052,-0.027885,0.013414,-0.041502,0.004074,0.002538,...,0.007505,-0.015248,-0.022396,0.006780,-0.029180,0.017866,0.030006,-0.008949,-0.016647,-0.009092
7021,0.200977,0.030445,-0.010369,0.093326,0.052145,0.010077,-0.014446,0.009902,0.099910,-0.056682,...,-0.035928,-0.002637,0.005529,0.021639,-0.000203,-0.051863,-0.016426,0.057054,-0.000361,0.029072


In [58]:
posts = pd.concat((posts, pd.get_dummies(posts['topic'], dtype = int, drop_first=True)), axis=1)
posts = posts.drop(['topic', 'text'], axis=1)
preprocess_posts = pd.merge(
    SVD_dataset,
    posts,
    left_index=True,
    right_index=True,
    how = 'outer'
)
#preprocess_posts = preprocess_posts.set_index('post_id', drop = True)

In [9]:
feed_data_bs_collab

,user_id,post_id,timestamp,target
0,200,1773,2021-12-29 15:24:59,0
1,200,2213,2021-12-29 15:24:31,1
2,200,1122,2021-12-29 15:23:54,0
3,200,1362,2021-12-29 15:23:29,0
4,200,1541,2021-12-29 15:21:53,0
...,...,...,...,...
3264095,168552,1823,2021-12-23 10:29:21,0
3264096,168552,5258,2021-12-23 10:29:11,0
3264097,168552,7063,2021-12-23 10:28:48,0
3264098,168552,6793,2021-12-23 10:27:50,0


In [6]:
# Можем приступать к коллаборации - фактически достаточно только таблицы feed_data

time_sort = feed_data_bs_collab.sort_values('timestamp')
train = time_sort.iloc[:-600000].copy()
test = time_sort.iloc[-600000:].copy()

In [27]:
train.sort_values('timestamp')

,user_id,post_id,timestamp,target
1905959,99521,1046,2021-10-27 16:20:42,0
545939,27511,6651,2021-10-27 16:20:42,0
681959,34319,3648,2021-10-27 16:20:42,0
273899,13904,6258,2021-10-27 16:20:42,0
2313959,121027,7134,2021-10-27 16:20:42,0
...,...,...,...,...
2288522,119756,630,2021-12-28 20:42:34,0
1064522,56945,5418,2021-12-28 20:42:34,0
520482,26238,1133,2021-12-28 20:42:34,0
1336522,70616,2057,2021-12-28 20:42:34,0


In [12]:
active_users = train['user_id'].value_counts().head(1000).index
train_simple = train[train['user_id'].isin(active_users)]

pivot_sample = train_simple.pivot_table(
    index = 'post_id',
    columns = 'user_id',
    values = 'target',
    fill_value = 0
).astype('float32')

if pivot_sample.shape[0] < 50000:
    corrs = pivot_sample.corr(method='pearson')

In [13]:
corrs

user_id,480,489,798,812,820,1050,1213,1306,1486,1850,...,167151,167158,167319,167350,167507,167829,167832,167836,167859,167926
user_id,,,,,,,,,,,,,,,,,,,,,
480,1.000000,NaN,-0.000478,-0.000390,-0.000478,-0.000276,NaN,NaN,NaN,-0.000276,...,-0.000390,NaN,-0.000276,-0.000390,NaN,-0.000390,-0.000478,-0.000618,-0.000276,-0.000478
489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
798,-0.000478,NaN,1.000000,-0.000390,-0.000478,-0.000276,NaN,NaN,NaN,-0.000276,...,-0.000390,NaN,-0.000276,-0.000390,NaN,-0.000390,-0.000478,-0.000618,-0.000276,-0.000478
812,-0.000390,NaN,-0.000390,1.000000,-0.000390,-0.000225,NaN,NaN,NaN,-0.000225,...,-0.000319,NaN,-0.000225,-0.000319,NaN,-0.000319,-0.000390,-0.000504,-0.000225,-0.000390
820,-0.000478,NaN,-0.000478,-0.000390,1.000000,-0.000276,NaN,NaN,NaN,-0.000276,...,-0.000390,NaN,-0.000276,-0.000390,NaN,-0.000390,-0.000478,-0.000618,-0.000276,-0.000478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167829,-0.000390,NaN,-0.000390,-0.000319,-0.000390,-0.000225,NaN,NaN,NaN,-0.000225,...,-0.000319,NaN,-0.000225,-0.000319,NaN,1.000000,-0.000390,-0.000504,-0.000225,-0.000390
167832,-0.000478,NaN,-0.000478,-0.000390,-0.000478,-0.000276,NaN,NaN,NaN,-0.000276,...,-0.000390,NaN,-0.000276,-0.000390,NaN,-0.000390,1.000000,-0.000618,-0.000276,-0.000478
167836,-0.000618,NaN,-0.000618,-0.000504,-0.000618,-0.000356,NaN,NaN,NaN,-0.000356,...,-0.000504,NaN,-0.000356,-0.000504,NaN,-0.000504,-0.000618,1.000000,-0.000356,-0.000618


In [14]:
corrs = (
    corrs
    .stack()
    .rename_axis(['userId1', 'userId2'])
    .reset_index()
)
corrs.columns = ['userId1', 'userId2', 'corr']

In [15]:
corrs = corrs[corrs['corr'] >= 0]
corrs

,userId1,userId2,corr
0,480,480,1.000000
12,480,3036,0.333015
65,480,14662,0.333015
272,480,64229,0.408021
713,798,798,1.000000
...,...,...,...
505365,167836,133303,0.447071
505372,167836,135147,0.315900
505517,167836,167836,1.000000
506230,167859,167859,1.000000


In [28]:
import pandas as pd
import numpy as np

# ПРЕДВАРИТЕЛЬНЫЕ ВЫЧИСЛЕНИЯ (делаем один раз, а не в цикле)
user_means = train.groupby('user_id')['target'].mean().to_dict()

# Создаем словарь соседей для быстрого доступа
neighbors_dict = {}
for _, row in corrs.iterrows():
    user = row['userId1']
    neighbor = row['userId2']
    corr_value = row['corr']
    
    if user not in neighbors_dict:
        neighbors_dict[user] = []
    neighbors_dict[user].append((neighbor, corr_value))

preds = []

# Ограничиваем уникальных пользователей для тестирования
# Для полного прогона уберите [100:] срез
test_users = test['user_id'].unique()  # Полный список
# test_users = test['user_id'].unique()[:100]  # Для теста первые 100 пользователей

for i, user in enumerate(test_users):
    # Прогресс-бар
    if i % 100 == 0:
        print(f"Обработано {i}/{len(test_users)} пользователей")
    
    if user in train['user_id'].unique():
        # Берем только нужные посты для предсказания
        user_test_posts = test[test['user_id'] == user]['post_id'].unique()
        
        # ИСПРАВЛЕНИЕ 1: правильно получаем соседей
        if user in neighbors_dict:
            neighbors = neighbors_dict[user]
            neighbor_ids = [n[0] for n in neighbors]
            corr_values = {n[0]: n[1] for n in neighbors}
        else:
            continue  # Пропускаем пользователей без соседей
            
        # ИСПРАВЛЕНИЕ 2: фильтруем train по соседям И по нужным постам
        train_part = train[
            (train['user_id'].isin(neighbor_ids)) & 
            (train['post_id'].isin(user_test_posts))
        ].copy()
        
        if train_part.empty:
            continue
            
        # ИСПРАВЛЕНИЕ 3: добавляем корреляции через map (быстрее чем merge)
        train_part['corr'] = train_part['user_id'].map(corr_values)
        train_part['neighbour_mean'] = train_part['user_id'].map(user_means)
        
        train_part['diff'] = train_part['target'] - train_part['neighbour_mean']
        train_part['diff_dot_corr'] = train_part['diff'] * train_part['corr']

        # Среднее текущего пользователя
        user_mean = user_means.get(user, 0)

        # Группируем по постам
        grouped = train_part.groupby('post_id')
        upper_part = grouped['diff_dot_corr'].sum()
        lower_part = grouped['corr'].sum()
        
        # Избегаем деления на ноль
        valid_posts = lower_part[lower_part != 0].index
        if len(valid_posts) == 0:
            continue
            
        predictions = (upper_part / lower_part + user_mean).reindex(valid_posts)
        predictions = predictions.reset_index()
        predictions.columns = ['post_id', 'prediction']
        predictions['user_id'] = user  # Исправлено: 'user_id' вместо 'userId'

        preds.append(predictions)

# Объединение ВНЕ цикла
if preds:
    preds_df = pd.concat(preds, ignore_index=True)
    
    # ИСПРАВЛЕНИЕ 4: правильное объединение с тестом
    preds_df = pd.merge(
        preds_df,
        test[['user_id', 'post_id', 'target']],
        on=['user_id', 'post_id'],
        how='left'
    )
else:
    preds_df = pd.DataFrame()

print(f"Готово! Получено {len(preds_df)} предсказаний")

Обработано 0/30480 пользователей
Обработано 100/30480 пользователей
Обработано 200/30480 пользователей
Обработано 300/30480 пользователей
Обработано 400/30480 пользователей
Обработано 500/30480 пользователей
Обработано 600/30480 пользователей
Обработано 700/30480 пользователей
Обработано 800/30480 пользователей
Обработано 900/30480 пользователей
Обработано 1000/30480 пользователей
Обработано 1100/30480 пользователей
Обработано 1200/30480 пользователей
Обработано 1300/30480 пользователей
Обработано 1400/30480 пользователей
Обработано 1500/30480 пользователей
Обработано 1600/30480 пользователей
Обработано 1700/30480 пользователей
Обработано 1800/30480 пользователей
Обработано 1900/30480 пользователей
Обработано 2000/30480 пользователей
Обработано 2100/30480 пользователей
Обработано 2200/30480 пользователей
Обработано 2300/30480 пользователей
Обработано 2400/30480 пользователей
Обработано 2500/30480 пользователей
Обработано 2600/30480 пользователей
Обработано 2700/30480 пользователей
Обра

In [19]:
def predict_user_based_collab(train, test, corrs, feed_data):
    """Оптимизированный User-Based Collaborative Filtering"""
    
    # 1. ПРЕДВАРИТЕЛЬНЫЕ ВЫЧИСЛЕНИЯ (один раз)
    # Средние оценки всех пользователей
    user_means = train.groupby('user_id')['target'].mean()
    
    # Словарь для быстрого доступа к соседям
    neighbors_dict = {}
    for _, row in corrs.iterrows():
        user = row['userId1']
        neighbor = row['userId2']
        corr_value = row['corr']
        
        if user not in neighbors_dict:
            neighbors_dict[user] = []
        neighbors_dict[user].append((neighbor, corr_value))
    
    predictions = []
    
    # 2. ОБРАБОТКА ТОЛЬКО НУЖНЫХ ПОЛЬЗОВАТЕЛЕЙ
    test_users = test['user_id'].unique()
    train_users = set(train['user_id'].unique())
    
    for user in test_users:
        # Пропускаем холодных пользователей
        if user not in train_users:
            continue
            
        # Получаем посты для предсказания
        user_posts = test[test['user_id'] == user]['post_id'].unique()
        
        # Получаем соседей из словаря
        if user not in neighbors_dict:
            continue
            
        neighbors = neighbors_dict[user]
        neighbor_ids = [n[0] for n in neighbors]
        neighbor_corrs = dict(neighbors)
        
        # 3. ВЫБОРКА ДАННЫХ СОСЕДЕЙ
        # Оценки соседей по релевантным постам
        neighbor_ratings = train[
            (train['user_id'].isin(neighbor_ids)) & 
            (train['post_id'].isin(user_posts))
        ].copy()
        
        if neighbor_ratings.empty:
            continue
            
        # Добавляем корреляции и средние
        neighbor_ratings['corr'] = neighbor_ratings['user_id'].map(neighbor_corrs)
        neighbor_ratings['neighbor_mean'] = neighbor_ratings['user_id'].map(user_means)
        
        # 4. ВЫЧИСЛЕНИЕ ПРЕДСКАЗАНИЙ
        # Формула: pred = mean_u + Σ[corr * (rating - mean_v)] / Σ|corr|
        neighbor_ratings['weighted_diff'] = (
            (neighbor_ratings['target'] - neighbor_ratings['neighbor_mean']) * 
            neighbor_ratings['corr']
        )
        
        # Группируем по постам
        grouped = neighbor_ratings.groupby('post_id')
        
        numerator = grouped['weighted_diff'].sum()
        denominator = grouped['corr'].apply(lambda x: x.abs().sum())
        
        # Избегаем деления на ноль
        valid_posts = denominator[denominator > 0].index
        
        if len(valid_posts) == 0:
            continue
            
        user_mean = user_means.get(user, 0)
        post_predictions = (numerator / denominator).reindex(valid_posts) + user_mean
        
        # 5. СОХРАНЕНИЕ РЕЗУЛЬТАТОВ
        user_preds = pd.DataFrame({
            'user_id': user,
            'post_id': post_predictions.index,
            'prediction': post_predictions.values
        })
        
        predictions.append(user_preds)
    
    # 6. ФИНАЛЬНОЕ ОБЪЕДИНЕНИЕ
    if predictions:
        final_preds = pd.concat(predictions, ignore_index=True)
        # Объединяем с реальными значениями из test
        final_preds = pd.merge(
            final_preds,
            test[['user_id', 'post_id', 'target']],
            on=['user_id', 'post_id'],
            how='left'
        )
        return final_preds
    else:
        return pd.DataFrame()

In [25]:
predictions

""
